In [1]:
import glob
import pandas as pd

def read_all_csvs_into_pandas(fpath:str)->pd.DataFrame:
    #Get the list of all csv files in the directory "fpath"
    csv_files = glob.glob(fpath)

    # Create an empty dataframe to store the combined data
    df_combined = pd.DataFrame()

    # Loop through each CSV file and append its contents to the combined dataframe
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df_combined = pd.concat([df_combined, df]).reset_index(drop=True)
    
    return df_combined

In [2]:
def transform_text_to_list(hist:str)->list[str]:
    return hist.\
        replace('\n', ' ').\
        replace("'", ' ').\
        replace("[", ' ').\
        replace("]", ' ').\
        replace(",", ' ').\
        strip().split()

In [3]:
def convert_type_of_all_list(l:list, dtype=int):
    return list(map(dtype,l))

In [4]:
FPATH_ITEMS = '../data/items_reduced/*.csv'
df_items = read_all_csvs_into_pandas(FPATH_ITEMS)

In [5]:
FPATH_TRAIN = '../data/train_reduced/*.csv'
df_train = read_all_csvs_into_pandas(FPATH_TRAIN)

In [6]:
df_train.head(1)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"


In [7]:
print(transform_text_to_list(df_train["timestampHistory"].iloc[0]))
print(convert_type_of_all_list(transform_text_to_list(df_train["timestampHistory"].iloc[0])))

['1657146417045', '1657146605778', '1657146698738']
[1657146417045, 1657146605778, 1657146698738]


In [8]:
def explode_columns(df:pd.DataFrame, cols_to_transform):
    for col in cols_to_transform:
        df[col] = df[col].apply(transform_text_to_list)
    return df.explode(cols_to_transform, ignore_index=True)

In [9]:
cols_to_explode = ["history", "numberOfClicksHistory", "timeOnPageHistory", "scrollPercentageHistory", "pageVisitsCountHistory", "userType"]
# cols_to_explode = ["history"]
cols_and_id = cols_to_explode.copy()
cols_and_id.insert(0,"userId")
cols_and_id = tuple(cols_and_id)
print(cols_and_id)

('userId', 'history', 'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory')


In [10]:
exploded_df = explode_columns(df_train.loc[:,cols_and_id], cols_to_explode)

In [11]:
exploded_df

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,c8aab885-433d-4e46-8066-479f40ba7fb2,76,20380,50.3,2
1,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,68d2039c-c9aa-456c-ac33-9b2e8677fba7,38,21184,18.18,1
2,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,13e423ce-1d69-4c78-bc18-e8c8f7271964,41,35438,16.46,1
3,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,3325b5a1-979a-4cb3-82b6-63905c9edbe8,7,6049,25.35,1
4,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,fe856057-f97d-419f-ab1c-97c5c3e0719c,80,210489,45.66,1
...,...,...,...,...,...,...
428356,c83b95712b4e2fd9bab7af5733d244680146355e2f8077...,f672d64f-6d2c-4c4c-8936-3a797683dd9e,0,50000,42.68,1
428357,5e1fe85101f303417768a5572f454da3dc15e645a85a12...,a289fde1-2ea9-4c18-a489-b8ad09c6a3dd,0,5492,8.18,1
428358,5e1fe85101f303417768a5572f454da3dc15e645a85a12...,e956323c-e672-4053-808a-c434b27cf613,1,38924,14.93,2
428359,5e1fe85101f303417768a5572f454da3dc15e645a85a12...,aa9d932a-7045-402d-80a9-431d1f17e0e8,1,17806,21.97,1


In [12]:
def check_df_size(df, exploded_df):
    return exploded_df.shape[0] == df["historySize"].sum().item()

In [13]:
check_df_size(df_train, exploded_df)

True

In [14]:

exploded_df['numberOfClicksHistory'] = exploded_df['numberOfClicksHistory'].astype('int32')
exploded_df['timeOnPageHistory'] = exploded_df['timeOnPageHistory'].astype('int64')
exploded_df['scrollPercentageHistory'] = exploded_df['scrollPercentageHistory'].astype('float32')
exploded_df['pageVisitsCountHistory'] = exploded_df['pageVisitsCountHistory'].astype('int32')

In [15]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428361 entries, 0 to 428360
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   428361 non-null  object 
 1   history                  428361 non-null  object 
 2   numberOfClicksHistory    428361 non-null  int32  
 3   timeOnPageHistory        428361 non-null  int64  
 4   scrollPercentageHistory  428361 non-null  float32
 5   pageVisitsCountHistory   428361 non-null  int32  
dtypes: float32(1), int32(2), int64(1), object(2)
memory usage: 14.7+ MB


In [16]:
exploded_df.isnull().sum()

userId                     0
history                    0
numberOfClicksHistory      0
timeOnPageHistory          0
scrollPercentageHistory    0
pageVisitsCountHistory     0
dtype: int64

In [17]:
exploded_df.userId.nunique()

30015

In [18]:
exploded_df.history.nunique()

56683

In [19]:
exploded_df["history"].value_counts().head(100)

history
d2593c3d-2347-40d9-948c-b6065e8459a9    1331
f6b5d170-48b9-4f8e-88d4-c84b6668f3bd    1148
1f32787b-de2b-49be-8c20-ddaeae34cc22    1019
6a83890a-d9e9-4f6b-a6c6-90d031785bbf    1017
f0a78e58-ec7e-494c-9462-fbd6446a9a89     960
                                        ... 
3936f6c7-3f6c-4205-8cda-3178160b6cd4     415
a8f0cda0-6c9c-45f6-b72a-3c20ba9624ea     415
8956c97a-6d85-4d4b-a6f7-6d42c6be97cc     414
b4a5680b-d0e1-4260-bc54-82fbe328d3a0     412
e22473a3-7a8a-46cb-8c86-1bda7a5c70de     412
Name: count, Length: 100, dtype: int64

In [20]:
comment_counts = pd.DataFrame(exploded_df["history"].value_counts())
rare_news = comment_counts[exploded_df["history"].value_counts() <= 30].index
common_news = exploded_df[~exploded_df["history"].isin(rare_news)]

In [22]:
print(exploded_df.shape[0], common_news.shape[0])

428361 283915


In [23]:
common_news["history"].value_counts().head(100000)

history
d2593c3d-2347-40d9-948c-b6065e8459a9    1331
f6b5d170-48b9-4f8e-88d4-c84b6668f3bd    1148
1f32787b-de2b-49be-8c20-ddaeae34cc22    1019
6a83890a-d9e9-4f6b-a6c6-90d031785bbf    1017
f0a78e58-ec7e-494c-9462-fbd6446a9a89     960
                                        ... 
f5ee9234-ae94-4799-97b4-b2f9ab383534      31
7bcce1b7-4b72-404b-bc0a-dbb60ce8d538      31
affc8e0c-2946-423b-b853-00755f082683      31
119c9753-fd82-422b-b6d1-e255cfbc5d42      31
bc9b4aa7-308b-42e4-8620-1eb25085c93a      31
Name: count, Length: 2463, dtype: int64

In [24]:
user_item_m = common_news.pivot(index='userId', columns='history', values='numberOfClicksHistory')

In [25]:
user_item_m

history,001b9bec-b1e2-4df2-9c43-8135627ce4c1,005a743a-0a81-4dd4-ba1f-6dc63cc33a34,00630ea8-4698-4748-9ef6-7c99f545a916,008d7a48-0864-4202-9a4e-b5d4f3c08e8b,00a396c3-bb4f-4fd7-9082-08e56925f2dc,00e7de24-edfe-44a9-8749-440ae0119af5,00f9aa60-3a6a-4e2e-bf5c-bb25f742cdf2,01514b11-4f32-49f8-abd3-00a8f753a0b0,01931947-bf19-490f-b085-61f335d31e89,01a50cd3-b862-4a88-85ee-5374533e1cc7,...,ff24f030-abbe-420a-b29e-de6794ec8fca,ff26f5dd-71d1-4364-940b-8647e048a209,ff3e8c61-d72a-42bf-a84a-ecf5409468dd,ff6e7987-c84d-4147-8de1-e8acf8a93df7,ff7421ee-fc6f-4c5d-846a-1143fae40832,ff872490-bef6-4c14-b5ec-66ac9d24ad19,ffabc550-b315-4070-94b6-713712da8ec2,ffe56be2-f822-44ac-b7c2-6bef6e8833a6,ffeea3a6-d969-4a9a-baf0-101da68da3f9,fff8f4aa-531e-4858-9108-31ec3c0f6ee6
userId,,,,,,,,,,,,,,,,,,,,,
0002727a78954d1ccf1655bd08084cf9419b8e18414bea91de32e19da1afc373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000e8aac7789ab9e01c7e45e2b5f8b528792b36aabf2ea519a934c9cbb22bc68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011b0b55d5dcd5dbd74b38dd6f79b7271ca49282c2ee8f48cc7183a0496b381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0026e502f6d599d6c57669cff722b0a4a10ab3c6341295ffdbf5d4d2595c60ef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002fbe018ce8d497330d1f8d363aeebeae6baba12894ae5439b79c0001acbb98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe733c3513bf64c819689f2a8cc4d68398b0fb0053c64868ef01b4075656836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe9909351d3e09950231e26867616f9c75725b3df622cb7cf5c3c10f9aa2d14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffecb603a24436d55cedaf64bf781cac2db5f60e10c1d3cdeaf600b4048da220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
